In [1]:
# 파일 읽기

import os, re, glob 
import cv2  #openCV 라이브러리 import하기
import numpy as np  
from sklearn.model_selection import train_test_split 

imagePath ='Outex-TC-00010/images'
categories = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12", "13", "14", "15", "16", "17", "18", "19", "20", "21", "22", "23"]
nb_classes = len(categories)

X = []  
Y = []  

for idx, cate in enumerate(categories):  
    label = [0 for i in range(nb_classes)]  
    label[idx] = 1
    image_dir = imagePath+'/'+cate+'/'
    
    for top, dir, f in os.walk(image_dir): 
        for filename in f:  
            img = cv2.imread(image_dir+filename,0)  
            X.append(img/256)  # 0 ~ 1사이로 정규화(Normalization)
            Y.append(label)

X = np.array(X)
Y = np.array(Y)

X_train, X_test, Y_train, Y_test = train_test_split(X,Y)  
xy = (X_train, X_test, Y_train, Y_test)

In [2]:
# LBP 특징 추출

import numpy as np
import cv2
from matplotlib import pyplot as plt
import pandas as pd

def thresholded(center, pixels):
    out = []
    for a in pixels:
        if a >= center:
            out.append(1)
        else:
            out.append(0)
    return out

def get_pixel_else_0(l, idx, idy, default=0):
    try:
        return l[idx,idy]
    except IndexError:
        return default

def run_LBP(img):
    for x in range(0, len(img)):
        for y in range(0, len(img[0])):
            center        = img[x,y]
            top_left      = get_pixel_else_0(img, x-1, y-1)
            top_up        = get_pixel_else_0(img, x, y-1)
            top_right     = get_pixel_else_0(img, x+1, y-1)
            right         = get_pixel_else_0(img, x+1, y )
            left          = get_pixel_else_0(img, x-1, y )
            bottom_left   = get_pixel_else_0(img, x-1, y+1)
            bottom_right  = get_pixel_else_0(img, x+1, y+1)
            bottom_down   = get_pixel_else_0(img, x,   y+1 )
            
            values = thresholded(center, [top_left, top_up, top_right, right, bottom_right,
                                          bottom_down, bottom_left, left])

            weights = [1, 2, 4, 8, 16, 32, 64, 128]
            res = 0
            for a in range(0, len(values)):
                res += weights[a] * values[a]

            transformed_img.itemset((x,y), res)

histList = []  #학습 데이터의 히스토그램 리스트 
for i in X_train:
    transformed_img = (i*256).astype('uint16')
    run_LBP(i)
    hist1,bins = np.histogram(transformed_img.flatten(),256,[0,256])
    histList.append(hist1.astype('uint16'))

histList2 = [] #훈련 데이터의 히스토그램 리스트 
for j in X_test:
    transformed_img = (j*256).astype('uint16')
    run_LBP(j)
    hist2,bins = np.histogram(transformed_img.flatten(),256,[0,256])
    histList2.append(hist2.astype('uint16'))
    
cv2.waitKey(0)
cv2.destroyAllWindows()

In [3]:
# 로지스틱 회귀
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 기울기 a와 절편 b의 값을 초기화 합니다.
a = np.zeros((256,nb_classes))
b = np.zeros(nb_classes)

#학습률을 정합니다.
lr = 0.01

#시그모이드 함수를 정의합니다.
def sigmoid(x):
    return 1 / (1 + np.exp(-x))


In [4]:
#경사 하강법을 실행
for i in range(30000):
    x_data = histList
    x_data = np.array(x_data)/256
    y_data = Y_train
    y_data = np.array(y_data)
    # 256X3240 dot ( 3240X256 * 256x10 = 100x10 ) = 256X10
    h = sigmoid(np.dot(x_data, a) + b                          # 가설 함수
    a_diff = (1/len(x_data)) * np.dot(x_data.T, (h - y_data))  # 예측 값과 라벨 값의 오차
    b_diff = (1/len(x_data)) * sum(h - y_data)
    a = a - lr * a_diff                                        # 가중치 및 편향 업데이트
    b = b - lr * b_diff

In [5]:
# 테스트셋 정확도 계산

count = 0

for i in range(len(Y_test)):
    # 1X256 * 256x10 = 1x10
    S = sigmoid(np.dot(histList2[i]/256, a) + b)
    S = list(S)
    S_P = S.index(max(S))
    Y_P = list(Y_test[i]).index(max(Y_test[i]))

    if S_P == Y_P:
        count += 1

result = count/len(Y_test)
print("최종 정인식률 : %.4f" % (result*100))

최종 정인식률 : 96.5741


In [6]:
# 경사 하강법을 실행합니다.
# 100개씩 배치 학습

# for i in range(1000):
#     for j in range(len(X_train)//100):
#         x_data = histList[0+100*j:100+100*j][:] # j: 0~8
#         x_data = np.array(x_data)/256
#         y_data = Y_train[0+100*j:100+100*j][:] # j: 0~8
#         y_data = np.array(y_data)
#         # 256X100 dot ( 100X256 * 256x10 = 100x10 ) = 256X10
#         a_diff = (1/len(x_data))*(np.dot(x_data.T, (sigmoid(np.dot(x_data, a) + b)) - y_data))
#         b_diff = (1/len(x_data))*sum(sigmoid(np.dot(x_data, a) + b) - y_data)
#         a = a - lr * a_diff
#         b = b - lr * b_diff